In [17]:
# pip install playwright

In [18]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

In [22]:
async def scrape_jobstreet():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # set to False to solve CAPTCHA manually
        page = await browser.new_page()
        await page.goto("https://ph.jobstreet.com/Data-Science-jobs?jobId=85753657&type=standard")

        print("🛑 Manually solve CAPTCHA in browser window if needed...")
        await asyncio.sleep(30)

        jobs = await page.query_selector_all('article[data-testid="job-card"]')
        print(f"🔍 Found {len(jobs)} job listings")

        data = []

        for job in jobs:
            try:
                title_el = await job.query_selector('[data-testid="job-title"]')
                company_el = await job.query_selector('[data-testid="company-name"]')
                location_el = await job.query_selector('[data-testid="job-location"]')
                salary_el = await job.query_selector('[data-testid="job-salary"]')
                posted_el = await job.query_selector('div[data-testid="job-card-date"]')
                url_el = await job.query_selector('a')
        
                title = await title_el.inner_text() if title_el else "N/A"
                company = await company_el.inner_text() if company_el else "N/A"
                location = await location_el.inner_text() if location_el else "N/A"
                salary = await salary_el.inner_text() if salary_el else "N/A"
                posted = await posted_el.inner_text() if posted_el else "N/A"
                job_url = await url_el.get_attribute("href") if url_el else "N/A"
        
                data.append({
                    "Title": title,
                    "Company": company,
                    "Location": location,
                    "Salary": salary,
                    "Posted": posted,
                    "URL": job_url
                })
        
            except Exception as e:
                print("⚠️ Error parsing a job:", e)

        await browser.close()
        return pd.DataFrame(data)

In [23]:
# Run the async function inside Jupyter
df = await scrape_jobstreet()
df.head()

🛑 Manually solve CAPTCHA in browser window if needed...
🔍 Found 32 job listings


,Title,Company,Location,Salary,Posted,URL
0,N/A,N/A,N/A,N/A,N/A,/job/85753657?type=standard&ref=search-standal...
1,N/A,N/A,N/A,N/A,N/A,/job/85795350?type=standard&ref=search-standal...
2,N/A,N/A,N/A,N/A,N/A,/job/85756366?type=standard&ref=search-standal...
3,N/A,N/A,N/A,N/A,N/A,/job/85782442?type=standard&ref=search-standal...
4,N/A,N/A,N/A,N/A,N/A,/job/85735664?type=standard&ref=search-standal...
